In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load
import os
import glob

### In this Notebook, I make predictions on a sample of over 6 million non-Wolf-Rayet stars from the dataset obtained from Gaia, which has undergone a series of filters as previously discussed.

In [2]:
xg_enn = load('modelos/xg_enn_100.joblib')
xg_tomek = load('modelos/xg_tomek_100.joblib')
xg_smt = load('modelos/xg_smt_100.joblib')

rf_enn = load('modelos/rf_enn_100.joblib')
rf_tomek = load('modelos/rf_tomek_100.joblib')
rf_smt = load('modelos/rf_smt_100.joblib')
muestra = pd.read_csv('archivos/Muestra final/muestra_completa_filtrada.csv')
muestra_entrenamiento = pd.read_csv('Archivos/Muestra//muestra_combinada.csv')
muestra_filtrada = muestra[~muestra['source_id'].isin(muestra_entrenamiento['source_id'])]

In [4]:


columnas_modelo = [
    'parallax', 'Gaia_G-Gaia_Bp', 'Gaia_G-Gaia_Rp', 'Gaia_Bp-Gaia_Rp',
    'Jmag-Hmag', 'Jmag-Kmag', 'Hmag-Kmag', 'W1-W2',
    'Gaia_G', 'Gaia_Bp', 'Gaia_Rp', 'Jmag', 'Hmag', 'Kmag', 'W1', 'W2'
]
columnas_extras = ['source_id', 'ra', 'dec']

ruta = 'Archivos/Muestra4/parquet/filtrados'
archivos = sorted(glob.glob(os.path.join(ruta, 'processed_*_filtrado.parquet')))

def procesar_archivo(filepath):
    df = pd.read_parquet(filepath, columns=columnas_extras + columnas_modelo)

    # Calcular probabilidades
    prob_enn = xg_enn.predict_proba(df[columnas_modelo])[:, 1]
    prob_tomek = xg_tomek.predict_proba(df[columnas_modelo])[:, 1]
    prob_smt = xg_smt.predict_proba(df[columnas_modelo])[:, 1]
    ponderacion = (prob_enn + prob_tomek + prob_smt) / 3

    df['prob_enn'] = prob_enn
    df['prob_tomek'] = prob_tomek
    df['prob_smt'] = prob_smt
    df['ponderacion'] = ponderacion

    df_filtrado = df[df['ponderacion'] >= 0.70]

    df_ordenado = df_filtrado.sort_values(by='ponderacion', ascending=False)

    return df_ordenado

resultados_totales = []  

for archivo in archivos:
    print(f"Procesando archivo: {archivo}")
    resultado_parcial = procesar_archivo(archivo)
    resultados_totales.append(resultado_parcial)

resultado_final = pd.concat(resultados_totales, ignore_index=True)

output_path = 'archivos/candidatos1/predicciones.csv'
resultado_final.to_csv(output_path, index=False)

print(resultado_final.head())
print(f"Shape final: {resultado_final.shape}")


Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_0_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_10_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_11_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_12_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_13_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_14_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_15_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_16_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_17_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_18_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados\processed_19_filtrado.parquet
Procesando archivo: Archivos/Muestra4/parquet/filtrados

In [5]:
print(f"Shape final: {resultado_final.shape}")
muestra_entrenamiento = pd.read_csv('Archivos/Muestra//muestra_combinada.csv')
muestra_filtrada = resultado_final[~resultado_final['source_id'].isin(muestra_entrenamiento['source_id'])]
muestra_filtrada.shape

Shape final: (289, 23)


C:\Users\diego\AppData\Local\Temp\ipykernel_5352\816653882.py:2: DtypeWarning: Columns (5,6,7,8,9,10,11,12,15,16,17,18,30,31,32,34,37,38,39,40,41,42,43,47,215,232,261,262,265,266,267,268,269,271,272,273,274,275,276,277,279,280,281,282,283,286,290,294,297,301,304,315,317,322,324) have mixed types. Specify dtype option on import or set low_memory=False.
  muestra_entrenamiento = pd.read_csv('Archivos/Muestra//muestra_combinada.csv')


(219, 23)

### 219 candidates, which is consistent with the rarity of Wolf-Rayet stars.

In [6]:
muestra_filtrada.to_csv('archivos/candidatos1/predicciones_sin_fp.csv', index=False)